In [13]:
import pandas as pd 
from dotenv import load_dotenv
import json
from openai import OpenAI
import os 
import numpy as np
import base64

def embed_chunk(df, i, client, window):
    window = df.iloc[i : i + window]
    ids = window['id'].values.tolist()

    try:
        result = client.embeddings.create(
            input=window['clean_text'].values.tolist(),
            model="text-embedding-3-small"
        )
    except Exception as e:
        print(e)
        return [
            {'id': x, 'embedding': None} 
            for x in ids
        ]

    return [
        {
            'id': ids[i], 
            'embedding': base64.b64encode(
                np.array(
                    x.embedding
                ).tobytes()
            ).decode('utf-8'), 
            'model': "text-embedding-3-small"
        } 
        for i, x in enumerate(result.data)
    ]
    

def embed_df(df, client):
    WINDOW = 1000

    embeddings = []
    i = 0
    while i + WINDOW < len(df):
        embeddings.extend(embed_chunk(df, i, client, WINDOW))
        i += WINDOW

    if i < len(df):
        embeddings.extend(embed_chunk(df, i, client, WINDOW))

    del(df)

    with open('data/embeddings.jsonl', 'a', encoding='utf-8') as file:
        for row in embeddings:
            file.write(json.dumps(row) + '\n')

    del(embeddings)


In [14]:
BUFFER_SIZE = 10000

load_dotenv()

client = OpenAI(api_key=os.environ.get("API_KEY"))

i = 0

with open('data/cleaned.jsonl', 'r', encoding='utf-8') as file:
    buffer = []
    for line in file:
        buffer.append(json.loads(line))
        if len(buffer) >= BUFFER_SIZE:
            buffer = pd.DataFrame(buffer)
            i += 1
            print(f"Processed buffer #{i}")
            embed_df(buffer, client)
            buffer = []

if len(buffer) >= 0:
    buffer = pd.DataFrame(buffer)
    i += 1
    print(f"Processed buffer #{i}")
    embed_df(buffer, client)
    buffer = []

Processed buffer #1
Processed buffer #2
Processed buffer #3
Processed buffer #4
Processed buffer #5
Processed buffer #6
Processed buffer #7
Processed buffer #8
Processed buffer #9
Processed buffer #10
Processed buffer #11
Processed buffer #12
Processed buffer #13
Processed buffer #14
Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 64390 tokens (64390 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
Processed buffer #15
Processed buffer #16
Processed buffer #17
Processed buffer #18
Processed buffer #19
Processed buffer #20
Processed buffer #21
Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 18014 tokens (18014 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code':

In [15]:
# Decode:
# np.frombuffer(base64.b64decode(j['embedding']))
